In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
import re
from tqdm import tqdm

import spacy

from sklearn.metrics.pairwise import cosine_similarity

import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

# Prepare Model

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = "sentence-transformers/all-MiniLM-L6-v2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModel.from_pretrained(checkpoint).to(device)


def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(
        input_mask_expanded.sum(1), min=1e-9)

def get_embeddings(documents, device=device):
    encoded_input = tokenizer(documents, 
                              padding=True, 
                              truncation=True, 
                              return_tensors='pt')
    if device.type == 'cuda':
        encoded_input = {k: v.to('cuda') for k, v in encoded_input.items()}

    with torch.no_grad():
        model_output = model(**encoded_input)

    sentence_embedding = mean_pooling(model_output, encoded_input['attention_mask'])

    if device.type == 'cuda':
        return sentence_embedding.cpu().numpy()
        
    return sentence_embedding.numpy()

C:\Users\Baha Tegar\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [3]:
# Load nlp model
nlp = spacy.load('en_core_web_sm')

# Load Dataset

In [4]:
df = pd.read_csv('GoodReads_100k_books.csv.gz', compression='gzip')

print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   author        100000 non-null  object 
 1   bookformat    96772 non-null   object 
 2   desc          93228 non-null   object 
 3   genre         89533 non-null   object 
 4   img           96955 non-null   object 
 5   isbn          85518 non-null   object 
 6   isbn13        88565 non-null   object 
 7   link          100000 non-null  object 
 8   pages         100000 non-null  int64  
 9   rating        100000 non-null  float64
 10  reviews       100000 non-null  int64  
 11  title         99999 non-null   object 
 12  totalratings  100000 non-null  int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 9.9+ MB
None


,author,bookformat,desc,genre,img,isbn,isbn13,link,pages,rating,reviews,title,totalratings
0,Laurence M. Hauptman,Hardcover,Reveals that several hundred thousand Indians ...,"History,Military History,Civil War,American Hi...",https://i.gr-assets.com/images/S/compressed.ph...,002914180X,9.78E+12,https://goodreads.com/book/show/1001053.Betwee...,0,3.52,5,Between Two Fires: American Indians in the Civ...,33
1,"Charlotte Fiell,Emmanuelle Dirix",Paperback,Fashion Sourcebook - 1920s is the first book i...,"Couture,Fashion,Historical,Art,Nonfiction",https://i.gr-assets.com/images/S/compressed.ph...,1906863482,9.78E+12,https://goodreads.com/book/show/10010552-fashi...,576,4.51,6,Fashion Sourcebook 1920s,41
2,Andy Anderson,Paperback,The seminal history and analysis of the Hungar...,"Politics,History",https://i.gr-assets.com/images/S/compressed.ph...,948984147,9.78E+12,https://goodreads.com/book/show/1001077.Hungar...,124,4.15,2,Hungary 56,26
3,Carlotta R. Anderson,Hardcover,"""All-American Anarchist"" chronicles the life a...","Labor,History",https://i.gr-assets.com/images/S/compressed.ph...,814327079,9.78E+12,https://goodreads.com/book/show/1001079.All_Am...,324,3.83,1,All-American Anarchist: Joseph A. Labadie and ...,6
4,Jean Leveille,NaN,"Aujourdâ€™hui, lâ€™oiseau nous invite Ã sa ta...",NaN,https://i.gr-assets.com/images/S/compressed.ph...,2761920813,NaN,https://goodreads.com/book/show/10010880-les-o...,177,4.00,1,Les oiseaux gourmands,1


In [5]:
# Handling empty data

df = df.fillna("").rename({'desc': 'description'}, axis=1)[:10_000]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   author        10000 non-null  object 
 1   bookformat    10000 non-null  object 
 2   description   10000 non-null  object 
 3   genre         10000 non-null  object 
 4   img           10000 non-null  object 
 5   isbn          10000 non-null  object 
 6   isbn13        10000 non-null  object 
 7   link          10000 non-null  object 
 8   pages         10000 non-null  int64  
 9   rating        10000 non-null  float64
 10  reviews       10000 non-null  int64  
 11  title         10000 non-null  object 
 12  totalratings  10000 non-null  int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 1015.8+ KB


In [6]:
# Item-to-item recommendation system based description only.

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # Lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # Tokenize document
    tokens = nltk.word_tokenize(doc)
    # Filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # Re-create docuement from filtered tokens
    doc = ' '.join(filtered_tokens)
    
    return doc

# Assuming `df['description']` is the column with text data
descriptions = list(df['description'])

# Use tqdm to show progress bar while normalizing
norm_corpus = []
for doc in tqdm(descriptions, desc="Normalizing descriptions"):
    norm_corpus.append(normalize_document(doc))

print(f"Total normalized documents: {len(norm_corpus)}")

Normalizing descriptions: 100%|█████████████████████████████████████████████████| 10000/10000 [00:12<00:00, 784.99it/s]

Total normalized documents: 10000


# Semantic Based Recommendation

In [12]:
# Extract important words

def preprocessing(text):
    # Get token of words
    doc = nlp(text)
    result = []
    for token in doc:
        # If the token is adjective, noun, propn, or verb
        if token.pos_ in ['NOUN', 'PROPN', 'VERB', 'ADJ']:
            result.append(token.text)
        else:
            continue
    return " ".join(result)

important_corpus = []
for doc in tqdm(descriptions, desc="Extract Important Word Descriptions"):
    important_corpus.append(preprocessing(doc))

print(f"Total important documents: {len(important_corpus)}")

Extract Important Word Descriptions: 100%|███████████████████████████████████████| 10000/10000 [07:21<00:00, 22.64it/s]

Total important documents: 10000


In [93]:
corpus = [important_corpus, norm_corpus]
books_list = df['title'].values

titles = ["The Fashion World of Jean Paul Gaultier: From the Sidewalk to the Catwalk",
          "The Business of Fashion: Designing, Manufacturing and Marketing",
          "Pattern-drafting for Fashion: The Basics: The Basics",
          "The Art of Dress: Fashion in England and France 1750 to 1820",
          "Fashion Flair for Portrait and Wedding Photography",
          "Graphic Design for Fashion",
          "Figure Drawing for Fashion Design",
          "Chronicle of Western Fashion",
          "Vintage Fashion",
          "Icons of Fashion: The 20th Century",
         ]

sample = "Fashion Sourcebook 1920s"
scores = []

for c in corpus:
    s = []
    for title in titles:
        desc_1 = c[np.where(books_list==sample)[0][0]]
        desc_2 = c[np.where(books_list==title)[0][0]]
        scoring = cosine_similarity(get_embeddings(desc_1), get_embeddings(desc_2))[0][0]
        s.append(scoring)
    scores.append(s)

In [94]:
pd.DataFrame(np.array(scores).T, columns=['important', 'normal'], index=titles)

,important,normal
The Fashion World of Jean Paul Gaultier: From the Sidewalk to the Catwalk,0.614356,0.680296
"The Business of Fashion: Designing, Manufacturing and Marketing",0.519874,0.485300
Pattern-drafting for Fashion: The Basics: The Basics,0.625809,0.622278
The Art of Dress: Fashion in England and France 1750 to 1820,0.649171,0.637237
Fashion Flair for Portrait and Wedding Photography,0.504274,0.447996
Graphic Design for Fashion,0.591846,0.606061
Figure Drawing for Fashion Design,0.587002,0.542397
Chronicle of Western Fashion,0.674326,0.689146
Vintage Fashion,0.644016,0.626385
Icons of Fashion: The 20th Century,0.812408,0.779049


**Testing with Random Books**

In [95]:
# Base Reference
idx_sample = np.where(books_list==sample)[0][0]
df.iloc[idx_sample]

author                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     Charlotte Fiell,Emmanuelle Dirix
bookformat                                                                                                                                  

In [96]:
base = important_corpus[np.where(books_list==sample)[0][0]]
base

'Fashion Sourcebook 1920s first book brand new series Fiell Publishing documents seasonal fashion styles 20th century decade decade illustrated original photographs drawings prints title reference work students fashion fashionistas Fashion Sourcebook 1920s focuses Art Deco period beautiful beaded dresses cloche hats t bar shoes worn fashionable flappers bright young things time accompanying introduction outlines major themes fashion period introduces famous designers assesses creative contributions Text English French German Available Fashion Sourcebook ISBN'

In [97]:
# test = "Victorian high society’s most daring equestrienne finds love and an unexpected ally in her fight for independence in the strong arms of London’s most sought after and devastatingly handsome half-Indian tailor. Evelyn Maltravers understands exactly how little she’s worth on the marriage mart. As an incurable bluestocking from a family tumbling swiftly toward ruin, she knows she’ll never make a match in a ballroom. Her only hope is to distinguish herself by making the biggest splash in the one sphere she excels: on horseback. In haute couture. But to truly capture London’s attention she’ll need a habit-maker who’s not afraid to take risks with his designs—and with his heart. Half-Indian tailor Ahmad Malik has always had a talent for making women beautiful, inching his way toward recognition by designing riding habits for Rotten Row’s infamous Pretty Horsebreakers—but no one compares to Evelyn. Her unbridled spirit enchants him, awakening a depth of feeling he never thought possible. But pushing boundaries comes at a cost and not everyone is pleased to welcome Evelyn and Ahmad into fashionable society. With obstacles spanning between them, the indomitable pair must decide which hurdles they can jump and what matters most: making their mark or following their hearts?"
test = "Without calculus, we wouldn’t have cell phones, TV, GPS, or ultrasound. We wouldn’t have unraveled DNA or discovered Neptune or figured out how to put 5,000 songs in your pocket. Though many of us were scared away from this essential, engrossing subject in high school and college, Steven Strogatz’s brilliantly creative, down‑to‑earth history shows that calculus is not about complexity; it’s about simplicity. It harnesses an unreal number—infinity—to tackle real‑world problems, breaking them down into easier ones and then reassembling the answers into solutions that feel miraculous. Infinite Powers recounts how calculus tantalized and thrilled its inventors, starting with its first glimmers in ancient Greece and bringing us right up to the discovery of gravitational waves (a phenomenon predicted by calculus). Strogatz reveals how this form of math rose to the challenges of each age: how to determine the area of a circle with only sand and a stick; how to explain why Mars goes “backwards” sometimes; how to make electricity with magnets; how to ensure your rocket doesn’t miss the moon; how to turn the tide in the fight against AIDS. As Strogatz proves, calculus is truly the language of the universe. By unveiling the principles of that language, Infinite Powers makes us marvel at the world anew."
test = preprocessing(test)
test

'calculus have cell phones TV GPS ultrasound unraveled DNA discovered Neptune figured put songs pocket many scared essential engrossing subject high school college Steven Strogatz creative down‑to‑earth history shows calculus complexity ’s simplicity harnesses unreal number infinity tackle real‑world problems breaking easier ones reassembling answers solutions feel miraculous Infinite Powers recounts calculus tantalized thrilled inventors starting first glimmers ancient Greece bringing discovery gravitational waves phenomenon predicted calculus Strogatz reveals form math rose challenges age determine area circle only sand stick explain Mars goes make electricity magnets ensure rocket miss moon turn tide fight AIDS Strogatz proves calculus language universe unveiling principles language Infinite Powers makes marvel world'

In [98]:
cosine_similarity(get_embeddings(base), get_embeddings(test))[0][0]

0.11722379

**Adding Genres**

In [99]:
corpus = [df['genre'].values]
books_list = df['title'].values

titles = ["The Fashion World of Jean Paul Gaultier: From the Sidewalk to the Catwalk",
          "The Business of Fashion: Designing, Manufacturing and Marketing",
          "Pattern-drafting for Fashion: The Basics: The Basics",
          "The Art of Dress: Fashion in England and France 1750 to 1820",
          "Fashion Flair for Portrait and Wedding Photography",
          "Graphic Design for Fashion",
          "Figure Drawing for Fashion Design",
          "Chronicle of Western Fashion",
          "Vintage Fashion",
          "Icons of Fashion: The 20th Century",
         ]

sample = "Fashion Sourcebook 1920s"
scores_g = []
genres = []

for c in corpus:
    s = []
    for title in titles:
        gen_1 = c[np.where(books_list==sample)[0][0]]
        gen_2 = c[np.where(books_list==title)[0][0]]
        genres.append(gen_2)
        scoring = cosine_similarity(get_embeddings(gen_1), get_embeddings(gen_2))[0][0]
        s.append(scoring)
    scores_g.append(s)

In [101]:
pd.set_option('display.max_colwidth', 2000)

print("BASE GENRE: ", gen_1)
pd.DataFrame({'genres': genres, 'score': np.array(scores_g).reshape(1, -1)[0]}, index=titles)

BASE GENRE:  Couture,Fashion,Historical,Art,Nonfiction


,genres,score
The Fashion World of Jean Paul Gaultier: From the Sidewalk to the Catwalk,"Couture,Fashion,Art",0.774646
"The Business of Fashion: Designing, Manufacturing and Marketing","Couture,Fashion",0.662918
Pattern-drafting for Fashion: The Basics: The Basics,"Crafts,Sewing,Couture,Fashion",0.531064
The Art of Dress: Fashion in England and France 1750 to 1820,"Couture,Fashion,History,Literature,18th Century,Nonfiction,Art,Art History,Cultural,France,European Literature,British Literature,Literature,19th Century,Art",0.756652
Fashion Flair for Portrait and Wedding Photography,"Art,Photography,Nonfiction",0.782738
Graphic Design for Fashion,"Couture,Fashion,Design,Nonfiction",0.951263
Figure Drawing for Fashion Design,"Couture,Fashion,Art",0.774646
Chronicle of Western Fashion,"Couture,Fashion,History,Nonfiction,Reference,Reference,Research",0.911028
Vintage Fashion,"Couture,Fashion,Nonfiction,History,Art,Reference",0.948082
Icons of Fashion: The 20th Century,"Couture,Fashion,Nonfiction",0.943938


In [108]:
temp = pd.DataFrame({'genres': genres,
                     'score_desc': np.array(scores[0]).reshape(1, -1)[0],
                     'score_gen': np.array(scores_g).reshape(1, -1)[0]},
                    index=titles)

temp['score_ovl'] = 0.75 * temp['score_desc'] + 0.25 * temp['score_gen']
temp.sort_values(by='score_desc', ascending=False)

,genres,score_desc,score_gen,score_ovl
Icons of Fashion: The 20th Century,"Couture,Fashion,Nonfiction",0.812408,0.943938,0.845290
Chronicle of Western Fashion,"Couture,Fashion,History,Nonfiction,Reference,Reference,Research",0.674326,0.911028,0.733502
The Art of Dress: Fashion in England and France 1750 to 1820,"Couture,Fashion,History,Literature,18th Century,Nonfiction,Art,Art History,Cultural,France,European Literature,British Literature,Literature,19th Century,Art",0.649171,0.756652,0.676041
Vintage Fashion,"Couture,Fashion,Nonfiction,History,Art,Reference",0.644016,0.948082,0.720033
Pattern-drafting for Fashion: The Basics: The Basics,"Crafts,Sewing,Couture,Fashion",0.625809,0.531064,0.602123
The Fashion World of Jean Paul Gaultier: From the Sidewalk to the Catwalk,"Couture,Fashion,Art",0.614356,0.774646,0.654429
Graphic Design for Fashion,"Couture,Fashion,Design,Nonfiction",0.591846,0.951263,0.681701
Figure Drawing for Fashion Design,"Couture,Fashion,Art",0.587002,0.774646,0.633913
"The Business of Fashion: Designing, Manufacturing and Marketing","Couture,Fashion",0.519874,0.662918,0.555635
Fashion Flair for Portrait and Wedding Photography,"Art,Photography,Nonfiction",0.504274,0.782738,0.573890
